In [1]:
import re
import requests

regex = r",\"([a-zA-Z0-9]{9})\","

r = requests.get("https://www.europeana.eu/en")
print(r.status_code)
#print(r.text)
if r.status_code != 200:
    exit(1)

matches = re.finditer(regex, r.text, re.MULTILINE)
potential_token = []
for matchNum, match in enumerate(matches, start=1):
    print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))
    for groupNum in range(0, len(match.groups())):
        groupNum = groupNum + 1
        print ("Group {groupNum} found at {start}-{end}: {group}".format(groupNum = groupNum, start = match.start(groupNum), end = match.end(groupNum), group = match.group(groupNum)))
        print(match.group(groupNum))
        potential_token.append(match.group(groupNum))
print(potential_token)

# find potential token from html

200
Match 1 was found at 133810-133823: ,"nLbaXYaiH",
Group 1 found at 133812-133821: nLbaXYaiH
nLbaXYaiH
Match 2 was found at 133932-133945: ,"RbWPAapQ6",
Group 1 found at 133934-133943: RbWPAapQ6
RbWPAapQ6
['nLbaXYaiH', 'RbWPAapQ6']


In [6]:
token = None

for tok in potential_token:
    url = f'https://api.europeana.eu/record/search.json?wskey={tok}&page=1&view=grid&query=test&profile=minimal&rows=1&start=1'
    r = requests.get(url)
    print(tok, r.status_code)
    if r.status_code == 200:
        token = tok
        break

nLbaXYaiH 200


In [37]:
keyword = "forêt de rambouillet"

In [38]:
initurl = f"https://api.europeana.eu/record/search.json?wskey={token}&page=1&view=grid&query={keyword}&qf=contentTier%3A%281%20OR%202%20OR%203%20OR%204%29&profile=minimal&rows=1&start=1"

In [39]:
r = requests.get(initurl)
if r.status_code != 200:
    exit(1)

In [40]:
datainit = r.json()
if datainit["success"] == "False":
    print("no result")
    exit(1)
totalc = datainit["totalResults"]
print(f"total result is {totalc}")

total result is 158


In [41]:
import time
result = []
index = 1

MAX_SEARCH = 100
while len(result) < totalc:
    searchurl = f"https://api.europeana.eu/record/search.json?wskey={token}&page=1&view=grid&query={keyword}&qf=contentTier%3A%281%20OR%202%20OR%203%20OR%204%29&profile=minimal&rows={MAX_SEARCH}&start={index}"

    r = requests.get(searchurl)
    if r.status_code != 200:
        print(r.status_code)
        print(r.text)
        exit(2)
    data = r.json()
    print(f"retrive {data['itemsCount']} items")
    index += data["itemsCount"]
    result += data["items"]

    time.sleep(0.1)

print(len(result))

retrive 100 items
retrive 58 items
158


In [42]:
import json
with open("europea.json", "w") as f:
    json.dump(result, f, indent=4)